In [2]:
import numpy as np
import pandas as pd
import matplotlib
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score


## Read the data

In [3]:
Cust_data=pd.read_csv('Customers.csv')
Prod_data=pd.read_csv('Products.csv')
Trans_data=pd.read_csv('Transactions.csv')

## Load and Aggregate Data based on requirements

In [4]:

# Merge datasets
transactions = pd.merge(Trans_data, Prod_data, on="ProductID", how="left")
data = pd.merge(transactions, Cust_data, on="CustomerID", how="left")

# Aggregate transaction data for each customer
customer_features = data.groupby("CustomerID").agg(
        total_spent=( "TotalValue",'sum'),
        transaction_count=("TransactionID", "count"),
        distinct_products=("ProductID", "nunique")
    ).reset_index()
scaler = MinMaxScaler()
customer_features_scaled = scaler.fit_transform(customer_features.iloc[:, 1:])

## Clustering the data

In [5]:
kmeans = KMeans(n_clusters=4, random_state=42)
clusters = kmeans.fit_predict(customer_features_scaled)

# Evaluate clustering using Davies-Bouldin Index
db_index = davies_bouldin_score(customer_features_scaled, clusters)

In [6]:
customer_features["Cluster"] = clusters

In [7]:
customer_features

,CustomerID,total_spent,transaction_count,distinct_products,Cluster
0,C0001,3354.52,5,5,3
1,C0002,1862.74,4,4,1
2,C0003,2725.38,4,4,1
3,C0004,5354.88,8,8,0
4,C0005,2034.24,3,3,1
...,...,...,...,...,...
194,C0196,4982.88,4,3,1
195,C0197,1928.65,3,3,1
196,C0198,931.83,2,2,2
197,C0199,1979.28,4,4,1


In [8]:
db_index

np.float64(0.7237774242163285)